# Evolve-Filter, time dep, N varies

* $\delta = 1/nx$
* $a = a_D$

In [5]:
notes = "\n Evolve then Filter: BDF2 using P2 and halving timestep at each refinement level \n\n"

from dolfin import *
from __future__ import print_function
import numpy as np
import sympy as sym
import csv

def compute_errors(u_e, u, t, mesh):
	L2n = errornorm(u_e, u, norm_type='L2', degree_rise=3, mesh=mesh)
	H1n = errornorm(u_e, u, norm_type='H1', degree_rise=3, mesh=mesh)
	return L2n, H1n

def compute_extrema(u, t):
    maxval = np.amax(u.vector().get_local())
    minval = np.amin(u.vector().get_local())
    return maxval, minval

# filter solving

def deconv_filter(u_, delta, Q, boundary):
    u_tilde = TrialFunction(Q)
    v = TestFunction(Q)

    F_Hfilter = v*u_tilde*dx + delta*delta*dot(grad(v), grad(u_tilde))*dx - v*u_*dx 

    a_Hfilter = lhs(F_Hfilter)
    L_Hfilter = rhs(F_Hfilter)
    
    bc = DirichletBC(Q, u_, boundary)
    
    A_Hfilter = assemble(a_Hfilter)
    bc.apply(A_Hfilter)

    b_Hfilter = assemble(L_Hfilter)
    bc.apply(b_Hfilter)

    solver = LUSolver(A_Hfilter)
    u_tilde = Function(Q)
    solver.solve(u_tilde.vector(), b_Hfilter)
    
    return u_tilde

def EF_t(folder, remark, N, delta, nx, dt, T, u_code, f_code, sigma, mu, velocity, chi):
    degree = 2
    
    t = 0.0
    u_exact = Expression(u_code, degree = degree+1, t = t)
    f = Expression(f_code, degree = degree+1, t = t)

    mesh = UnitSquareMesh(nx,nx)
#     mesh = RectangleMesh(Point(-5.0,-5.0), Point(5.0, 5.0), 5*nx, 5*nx)
    Q = FunctionSpace(mesh, "CG", degree)

    # Set up boundary condition
    u_D = Expression(u_exact.cppcode, degree = degree+1, t = t)
    
    def boundary(x, on_boundary):
        return on_boundary

    
# ------------ STEP 1: Solve on coarse grid ----------------------#

    # Test and trial functions
    u, v = TrialFunction(Q), TestFunction(Q)
    u_n0 = interpolate(u_D, Q)
    u_n1 = Function(Q)
    u_ = Function(Q)
    

    
    # Galerkin variational problem (BDF2)
    F = (1.5*u - 2.0*u_n1 + 0.5*u_n0)*v*dx
    F += dt*(mu*dot(grad(v), grad(u))*dx + v*dot(velocity, grad(u))*dx + sigma*v*u*dx - f*v*dx)
    

    # Create bilinear and linear forms
    a1 = lhs(F)
    L = rhs(F)


    # Create progress bar
    progress = Progress('Time-stepping')
    set_log_level(PROGRESS)

    # Outputting files
    #out_file_uexact = File(folder+"evolve_u_exact_"+str(nx)+".pvd") 
    #out_file_ubar = File(folder+str(mu)+"evolve_u_"+str(nx)+".pvd") 
    ue = interpolate(u_exact, Q)
    out_file_ind = File(folder+"EF_aD"+str(nx)+'_N'+str(N)+".pvd")
    out_file_ufilter = File(folder+"EF_u_"+str(nx)+'_N'+str(N)+".pvd") 

    # Save t = 0.0
    #out_file_uexact << (ue, float(t))
    
    
    u_.rename('u','u')
    indicator = Expression('0.0', degree=2, t=0.0)
    indicator = interpolate(indicator, Q)
    indicator.rename('a','a')
    
    #out_file_ufilter << (u_, float(t))
    #out_file_ind << (indicator, float(t))
    
    
        # Define boundaries
    #inflow = 'near(x[0],0) && x[1]<=0.2 && x[1]>=0'		# x=0, 0<y<.2
    #no_inflow = 'near(x[0],0) && x[1]>0.2 && x[1]<=1.0'	# x=0, .2<y<1.0
    inflow = 'near(x[0],0)'		# x=0
    top = 'near(x[1],1)'		# y=1
    bottom = 'near(x[1],0)'		# y=0
    outflow = 'near(x[0],1)'	# x=1

    bc1 = DirichletBC(Q,Constant(0.0),outflow)
#     bc2 = DirichletBC(Q,Constant(0.0),inflow)
#     bc3 = DirichletBC(Q,Constant(0.0),top)
    bc4 = DirichletBC(Q,Constant(0.0),bottom)
#     bc5 = DirichletBC(Q,Constant(0.0),no_inflow)

    bcu = [bc1, bc4]
    
    while t - T + dt < DOLFIN_EPS:
        # Step 1 Solve on Coarse Grid
        t += dt

        u_D.t = t
        f.t = t
        u_exact.t = t
        velocity.t = t    
        
        # Assemble matrices
        A1 = assemble(a1)
        b = assemble(L)
        [bc.apply(A1) for bc in bcu]
        [bc.apply(b) for bc in bcu]
        
#         bc = DirichletBC(Q, u_D, boundary)
#         bc.apply(A1)
#         bc.apply(b)
        
        solve(A1, u_.vector(), b)#, 'gmres')
        progress.update(t / T)
        
#         if near(t,0.5):
#             errors = compute_errors(u_exact, u_, t, mesh)
#             print("Before Filter \n" + errors)

        #out_file_uexact << (ue, float(t))
        #out_file_ubar << (u_, float(t))
        
# ------------ STEP 2: Helmholtz filter ----------------------#
    
        if N == 0:
            u_tilde0 = deconv_filter(u_, delta, Q, boundary)
            DF = Expression('u_tilde0', degree = degree, u_tilde0 = u_tilde0)

        if N == 1:
            u_tilde0 = deconv_filter(u_, delta, Q, boundary)
            u_tilde1 = deconv_filter(u_tilde0, delta, Q, boundary)
            DF = Expression('2.0*u_tilde0 - u_tilde1', degree = degree, u_tilde0 = u_tilde0, u_tilde1 = u_tilde1)

        if N == 2:
            u_tilde1 = deconv_filter(u_, delta, Q, boundary)
            u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
            u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
            DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3)

        if N == 3:
            u_tilde1 = deconv_filter(u_, delta, Q, boundary)
            u_tilde2 = deconv_filter(u_tilde1, delta, Q, boundary)
            u_tilde3 = deconv_filter(u_tilde2, delta, Q, boundary)
            u_tilde4 = deconv_filter(u_tilde3, delta, Q, boundary)
            DF = Expression('3.0*u_tilde1 - 3.0*u_tilde2 + u_tilde3 + u_tilde1 - 3.0*u_tilde2 + 3.0*u_tilde3 - u_tilde4', degree = degree, u_tilde1 = u_tilde1, u_tilde2 = u_tilde2, u_tilde3 = u_tilde3, u_tilde4 = u_tilde4)
            
        # Compute the indicator function N = 0
        indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = DF)
        indicator = interpolate(indicator, Q)
        max_ind = np.amax(indicator.vector().get_local())

        if max_ind < 1:
            max_ind = 1.0

        indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
        indicator = interpolate(indicator, Q)

        indicator.rename('a','a')
        out_file_ind << (indicator, t)

        # Apply the filter
        u_bar = TrialFunction(Q)
        F_filter = v*u_bar*dx + delta*delta*dot(grad(v), indicator*grad(u_bar))*dx - v*u_*dx 

        a_filter = lhs(F_filter)
        L_filter = rhs(F_filter)

        A_filter = assemble(a_filter)
#         bc.apply(A_filter)
        [bc.apply(A_filter) for bc in bcu]

        b_filter = assemble(L_filter)
#         bc.apply(b_filter)
        [bc.apply(b_filter) for bc in bcu]
        solver = LUSolver(A_filter)
        u_bar = Function(Q)
        solver.solve(u_bar.vector(), b_filter)


#         out_file_ufilter << (u_bar, float(t))

        u_n0.assign(u_n1)
        u_n1.assign((1 - chi)*u_+ chi*u_bar)
        u_n1.rename('u','u')
        out_file_ufilter << (u_n1, float(t))
        
    out_file_ind << (indicator, float(t))
    out_file_ufilter << (u_bar, float(t))   
    L2, H1 = compute_errors(u_exact, u_n1, t, mesh)
    maxval, minval = compute_extrema(u_n1, t)
    
    print(nx,",",L2,",",H1,",",maxval,",",minval)
    return str(L2)+','+str(H1)+','+str(maxval)+','+str(minval)
    
# ------------------------------------------ #

from datetime import datetime, date

def run(nx,chi,dt,N,sigma,mu):
    delta = 1.0/nx;
    d_date = datetime.now()
    today = d_date.strftime('%Y-%m-%d')
    f = open(today+'_log_EFR_un1.txt',"a+")


    T = 6
    degree = 2
    velocity = Expression(('cos(t)','sin(t)'), degree = degree, t = 0)
    adr_f = Expression('exp(-(pow(x[0]-0.5,2)+pow(x[1]-0.5,2))/pow(0.07,2))', degree = degree)
    folder = 'results_EFR_BC/sigma'+str(sigma)+'_mu'+str(mu)
    sim_data = 'EFR, dt='+str(dt)+', T='+str(T)+', P'+str(degree)
    f.write(sim_data+'\n')


    now = d_date.strftime("%I:%M:%S %p")
    f.write('start: '+now+'\n\n')

    remark = ''
    output = EF_t(folder,remark, N, delta, nx, dt, T, '0', adr_f.cppcode, sigma, mu, velocity, chi)
    f.write(str(nx)+','+str(dt)+','+output+'\n')

    d_date = datetime.now()
    now = d_date.strftime("%I:%M:%S %p")
    f.write('\nend: '+now+'\n\n')
    f.write(notes)

    f.close()

In [6]:
dt = 0.1
N = 0
sigma = 0.1
mu = 0.00005
# nx,chi,dt,N
run(25,1,dt,N,sigma,mu)
# dt = dt/2
run(50,0.5,dt,N,sigma,mu)
# dt = dt/2

25 , 1.96985697951e+11 , 3.72360013476e+12 , 277874908557.0 , -3.61183560355e+12
50 , 3.24827083853e+39 , 6.19256559864e+40 , 5.38375006515e+40 , -1.67419812531e+39


In [7]:
dt = 0.1
N = 0
sigma = 0.1
mu = 0.00005
# nx,chi,dt,N
run(100,1,dt,N,sigma,mu)
# dt = dt/2
run(200,0.5,dt,N,sigma,mu)
# dt = dt/2

100 , 1.23843914451e+28 , 8.51733774752e+29 , 4.80345467387e+29 , -4.69279096366e+28
200 , 4.32886948871e+17 , 1.061728956e+20 , 2.47454714682e+18 , -1.93485678219e+19
